<h1>Simuler une loi de probabilité donnée</h1>

Comment faire pour simuler un générateur aléatoire obéissant à une certaine loi de probabilité $P(x)$ ?

<h2>Méthode 1: inversion de la fonction de répartition</h2>

On se donne une loi de probabilité $P(x)$ pour $x\in[-\infty,+\infty]$. 

On cherche la fonction de partition $F$ de la loi de probabilité $P$ définie par: $y=F(x)=\int_{-\infty}^x P(u).\textrm{d}u$.

<i>N.B. Si le support de la loi de probabilité est $[a,b]$, alors la fonction de partition est définie par: $y=F(x)=\int_{a}^x P(u).\textrm{d}u$.</i>

On calcule $x=F^{-1}(y)$.

Si $y$ suit la loi de probabilité uniforme (i.e. <tt>Random.float 1.</tt>), alors $x$ suit la loi de probabilité $P(x)$.

<h3>Exercice 1: loi exponentielle</h3>

But: inverser la loi de probabilité exponentielle, à support dans $\mathbb{R}^+$, de paramètre $\lambda$: $P(x)=\lambda.\exp(-\lambda.x)$

<b>1.</b> On définit la fonction $P(u)=\lambda.\exp(-\lambda.u)$.

<b>2.</b> On calcule la fonction de partition $y=F(x)=\int_{0}^xP(u).\textrm{d}u=1-e^{-\lambda.x}$

<b>3.</b> On calcule $x=F^{-1}(y)=\frac{-1}{\lambda}.\ln(1-y)$

<b>4.</b>Tracer l'histogramme de la loi de probabilité $F^{-1}(y)$ où $y$ est donné par la loi uniforme sur [0,1]. On fera 1000 tirages.

Comparer avec $P(x)$ pour différentes valeurs de $\lambda$: 0.1, 0.5, 0.9

In [1]:
open Random;;
Random.self_init;;
#use "topfind";;
#require "plplot";;
open Plplot;;
module P = Plot;;
let couleurs_list = [[ 0;255;255;255]; (*`white*)
                     [ 1;  0;  0;  0]; (*`black*)
                     [ 2;  0;  0;255]; (*`blue*)
                     [ 3;255;  0;  0]; (*`red*)
                     [ 4;165; 42; 42]; (*`brown*)
                     [ 5;  0;  0;  0]; [ 6;  0;  0;  0]; [ 7;  0;  0;  0]; [ 8;  0;  0;  0]; [ 9;  0;  0;  0]; 
                     [10;200;200;200]; (*`gray*)
                     [11;  0;255;255]; (*`light_blue*)
                     [12;  0;255;  0]; (*`green*)
                     [13;255;255;  0]; (*`yellow*)
                     [14;255;  0;255]; (*`pink*)
                     [15;160;  0;213]; (*`purple*) ]
let rec loop couleurs_list = match couleurs_list with
    | [n;r;g;b]::tl -> plscol0 n r g b; loop tl
    | _ -> ();;
let couleurs = (fun () -> plscolbg 255 255 255; loop couleurs_list)
let initialisation filename xmin xmax ymin ymax = 
        P.init (xmin, ymin) (xmax, ymax) `greedy (`svg `core) ~filename:(filename^".svg") ~pre:couleurs
let xlabel texte = P.text_outside `black (`bottom 0.5) 3. texte
let ylabel texte = P.text_outside `black (`left 0.5) 5. texte 
let label texte_x texte_y titre = P.label texte_x texte_y titre

- : unit = ()
Findlib has been successfully loaded. Additional directives:
  #require "package";;      to load a package
  #list;;                   to list the available packages
  #camlp4o;;                to load camlp4 (standard syntax)
  #camlp4r;;                to load camlp4 (revised syntax)
  #predicates "p,q,...";;   to set these predicates
  Topfind.reset();;         to force that packages will be reloaded
  #thread;;                 to enable threads

- : unit = ()


/usr/lib/ocaml/plplot: added to search path
/usr/lib/ocaml/plplot/plplot.cma: loaded


- : unit -> unit = <fun>

module P = Plplot.Plot

val couleurs_list : int list list =
 [[0; 255; 255; 255]; [1; 0; 0; 0]; [2; 0; 0; 255]; [3; 255; 0; 0];
 [4; 165; 42; 42]; [5; 0; 0; 0]; [6; 0; 0; 0]; [7; 0; 0; 0]; [8; 0; 0; 0];
 [9; 0; 0; 0]; [10; 200; 200; 200]; [11; 0; 255; 255]; [12; 0; 255; 0];
 [13; 255; 255; 0]; [14; 255; 0; 255]; [15; 160; 0; 213]]

...

val couleurs : unit -> unit = <fun>

val initialisation : string -> float -> float -> float -> float -> P.stream_t =
 <fun>

val xlabel : string -> P.plot_t = <fun>

val ylabel : string -> P.plot_t = <fun>

val label : string -> string -> string -> P.plot_t = <fun>

In [2]:
let bar ?x_width ?x_offset color xs ys = 
    let x_width_ = match x_width with
        |None -> 0.4
        |Some x -> x in
    let x_offset_ = match x_offset with
        |None -> 0.
        |Some x -> x in
    
    let x = Array.make (4*(Array.length xs)) 0. in
    let y = Array.make (4*(Array.length xs)) 0. in
    for i = 0 to Array.length xs - 1 do
        x.(4*i)   <- x_offset_ +. xs.(i) -. x_width_;
        x.(4*i+1) <- x_offset_ +. xs.(i) -. x_width_;
        x.(4*i+2) <- x_offset_ +. xs.(i) +. x_width_;
        x.(4*i+3) <- x_offset_ +. xs.(i) +. x_width_;
        y.(4*i)   <- 0.;
        y.(4*i+1) <- ys.(i);
        y.(4*i+2) <- ys.(i);
        y.(4*i+3) <- 0.;
    done; P.polygon ~fill:true color x y;;

val bar :
 ?x_width:float ->
 ?x_offset:float -> P.color_t -> float array -> float array -> P.plot_t =
 <fun>

In [3]:
let histogramme ?normalized valeurs limites =
    let normalized_ = match normalized with
        |None | Some false -> false (*somme bins=1*)
        |Some true -> true (*integrale bins d(limites) = 1 *) in
    let nbre_val = Array.length valeurs in
    let nbre_bin = Array.length limites - 1 in
    let bins = Array.make nbre_bin 0 in
    let rec loop num_val =
         if num_val < nbre_val 
         then begin
            let valeur = valeurs.(num_val) in
            let i=ref 0 in
            while !i < (nbre_bin-1) && limites.(!i+1)<valeur do incr i;done;
            if !i<nbre_bin && valeur<limites.(nbre_bin-1) then bins.(!i)<-bins.(!i)+1;
            loop (num_val+1);
        end in
     loop 0;
     let largeur_intervalle = 
         if normalized_ 
         then (float_of_int nbre_bin)/.(limites.(nbre_bin) -. limites.(0))/.(float_of_int nbre_val)
         else 1./.(float_of_int nbre_val) in 
     Array.init nbre_bin (fun i -> (float_of_int bins.(i))*.largeur_intervalle);;

val histogramme :
 ?normalized:bool -> float array -> float array -> float array = <fun>

In [4]:
let linspace debut fin nbre_pts =
     let step = (fin-.debut)/.(float_of_int (nbre_pts-1)) in
     let rec loop i acc =
         if i<nbre_pts
         then loop (i+1) ((debut+.(float_of_int i)*.step)::acc)
         else List.rev acc in
       loop 0 [];;

val linspace : float -> float -> int -> float list = <fun>

In [5]:
let f x lambda = lambda*.exp((-.lambda)*.x);;
let f_moins_un y lambda = (-.1.)/.lambda*.log(1.-.y);;

let trace filename lambda xmax =
    let data = Array.init (10*1000) (fun _ -> f_moins_un (Random.float 1.) lambda) in
    let limites = Array.of_list (linspace 0. xmax 51) in
    let xs = Array.of_list (linspace 0. xmax 50) in
    let ys = histogramme ~normalized:true data limites in
    let ys' = Array.map (fun x-> f x lambda) xs in
    let p = initialisation filename (-.xmax/.200.) (xmax+.xmax/.200.) 0. lambda in
    P.plot ~stream:p [bar ~x_width:(xmax/.200.) `blue xs ys;
                      P.lines `red xs ys';
                      label "x" "probabilité" ("Loi de proba λ.exp(-λ.x) avec λ="^(string_of_float lambda));
                      P.legend ~pos:(P.viewport_pos 0.5 0.)
                              [[P.line_legend "simulation" `blue];
                               [P.line_legend "théorie" `red]]];
    P.finish ~stream:p ();; 

val f : float -> float -> float = <fun>

val f_moins_un : float -> float -> float = <fun>

val trace : string -> float -> float -> unit = <fun>

In [6]:
trace "graph1" 0.1 50.

- : unit = ()

<img src="graph1.svg" width=500 />

In [7]:
trace "graph2" 0.5 10.

- : unit = ()

<img src="graph2.svg" width=500 />

In [8]:
trace "graph3" 0.9 5.

- : unit = ()

<img src="graph3.svg" width=500 />

<h3>Exercice 2: loi de Laplace</h3>

Faire de même avec la loi de Laplace: $P(x)=\frac12.e^{-|x|}$.

On tracera $P(x)$ sur $[-5,5]$

<b>1.</b> On définit la fonction $P(u)=\frac12.e^{-|u|}$.

<b>2.</b> On calcule la fonction de partition $y=F(x)=\int_{-\infty}^xP(u).\textrm{d}u$.<br>
Si $u<0$:  $P(u)=\frac12.e^{u}$,&nbsp;&nbsp; $y=F(x)=\int_{-\infty}^xP(u).\textrm{d}u=\frac12.e^{x}$<br>
Si $u>0$:  $P(u)=\frac12.e^{-u}$,&nbsp;&nbsp; $y=F(x)=\frac12+\int_{0}^xP(u).\textrm{d}u=\frac12+\frac12.(1-e^{-x})=1-\frac12.e^{-x}$

<b>3.</b> On calcule $x=F^{-1}(y)$<br>
Si $u<0$: $x=\ln(2.y)$<br>
Si $u>0$: $x=-\ln(2-2.y)$

In [9]:
let f x = exp(-.abs_float(x))/.2.;;
let f_moins_un y = 
    if y<0.5 (*en x=0, F(0)=1/2*)
    then log(2.*.y)
    else -.log(2.-.2.*.y) (*on pourrait simplifier en -log(2.*.y) *);;
    
let trace filename xmax =
    let data = Array.init (10*1000) (fun _ -> f_moins_un (Random.float 1.)) in
    let limites = Array.of_list (linspace (-.xmax) xmax 51) in
    let xs = Array.of_list (linspace (-.xmax) xmax 50) in
    let ys = histogramme ~normalized:true data limites in
    let ys' = Array.map (fun x-> f x) xs in
    let p = initialisation filename (-.xmax-.xmax/.200.) (xmax+.xmax/.200.) 0. 0.5 in
    P.plot ~stream:p [bar ~x_width:(xmax/.200.) `blue xs ys;
                      P.lines `red xs ys';
                      label "x" "probabilité" ("Loi de proba exp(-|x|)");
                      P.legend [[P.line_legend "simulation" `blue];
                                [P.line_legend "théorie" `red]]];
    P.finish ~stream:p ();; 

val f : float -> float = <fun>

val f_moins_un : float -> float = <fun>

val trace : string -> float -> unit = <fun>

In [10]:
trace "graph4" 5.

- : unit = ()

<img src="graph4.svg" width=500 />

<h4>Problème</h4>
Problème: il n'est pas toujours possible d'inverser la fonction $F$ (ex: pour la loi gaussienne: $P(x)=\frac{1}{\sqrt{2.\pi}}.\exp(-x^2/2)$, on a: $F(x)=\displaystyle\int_{-\infty}^x \frac{1}{\sqrt{2.\pi}}.\exp(-u^2/2).\textrm{d}u$ qui n'est pas inversable).

<h2>Méthode 2: algorithme de rejet</h2>

Le but de cet algorithme est de simuler un tirage suivant une loi de probabilité $P(x)$ à partir d'un générateur aléatoire suivant une loi de probabilité $g(x)$.

Exemple: à partir de $g(x)$ suivant la loi de Laplace (que l'on peut générer par la méthode d'inversion), simuler une loi de Gauss $P(x)=\frac{1}{\sqrt{2.\pi}}.\exp(-x^2/2)$ (que l'on ne peut pas générer par la méthode d'inversion).

L'algoritme consiste à:
<ol>
<li> simuler une variable aléatoire $X$ suivant la loi de probabilité $g(x)$
<li> simuler une variable aléatoire $U$ suivant la loi uniforme sur [0,1] (i.e. <tt>U=random.random()</tt>)
<li> <ul>
<li> si $U\leq\frac{P(X)}{M.g(x)}$ où $M$ est une contante, alors on accepte $X$ comme realisation de la variable aléatoire générée par la loi $P(X)$.
<li> sinon, on ré-itère depuis 1.
</ul>
</ol>

N.B. La constante $M$ doit être telle que: $\forall x$, $P(x)\leq M.g(x)$. On a intérêt (pour minimiser les cas de rejet) à prendre $M$ la plus petite possible.

<b>1.</b> A l'aide de la méthode d'inversion, créer un générateur suivant la loi de Laplace

In [11]:
let laplace()=
    let y=Random.float 1. in
    if y<0.5
    then log(2.*.y)
    else -.log(2.*.(1.-.y))

val laplace : unit -> float = <fun>

<b>2.</b> Dans le cas où $P(x)=\frac{1}{\sqrt{2.\pi}}.e^{-x^2/2}$ et $g(x)=\frac12.e^{-|x|}$, on peut montrer que la plus petite valeur de $M$ est $\sqrt{\frac{2.e}\pi}$.

Dans une fonction <tt>Gauss()</tt>: tirer une valeur de $x$ suivant la loi de probabilité $g(x)$, tirer une variable $u$ suivant la loi uniforme sur [0,1] et réitérer jusqu'à ce que $u\leq\frac{P(x)}{M.g(x)}$. Renvoyer alors la valeur de $x$.

In [12]:
let pi = 4. *. atan 1.;;

let gauss() =
    (*
    on veut Gauss
    on part de Laplace g(z)
    M=sqrt(2*e/pi)
    on tire au hasard, on rejete si proba > P(z)/(M*g(z))
    *)
    let m = sqrt(2.*.exp(1.)/.pi) in
    let p x = (*Gauss*)
        1./.sqrt(2.*.pi)*.exp(-.(x**2.)/.2.) in
    let g x = (*Laplace*)
        1./.2.*.exp(-.abs_float x) in
    let alpha x =
        (p x)/.(m*.(g x)) in
    
    let rec loop () =
        let x = laplace() in
        let u = Random.float 1. in
        if u<=(alpha x) then x else loop() in
    loop();;

val pi : float = 3.14159265358979312

val gauss : unit -> float = <fun>

<b>3.</b> Tirer au sort 10.000 valeurs en utilisant la fonction <tt>gauss()</tt>.

Réaliser l'histogramme et comparer à $P(x)$.

In [13]:
let trace filename xmax =
    let data = Array.init (10*1000) (fun _ -> gauss()) in
    let limites = Array.of_list (linspace (-.xmax) xmax 51) in
    let xs = Array.of_list (linspace (-.xmax) xmax 50) in
    let ys = histogramme ~normalized:true  data limites in
    let ys' = Array.map (fun x-> 1./.sqrt(2.*.pi)*.exp(-.(x**2.)/.2.)) xs in
    let p = initialisation filename (-.xmax-.xmax/.200.) (xmax+.xmax/.200.) 0. 0.5 in
    P.plot ~stream:p [bar ~x_width:(xmax/.200.) `blue xs ys;
                      P.lines `red xs ys';
                      label "x" "probabilité" ("Loi de proba exp(-x**2/2) / sqrt(2*pi)");
                      P.legend [[P.line_legend "simulation" `blue];
                                [P.line_legend "théorie" `red]]];
    P.finish ~stream:p ();; 

trace "graph5" 5.;;

val trace : string -> float -> unit = <fun>

- : unit = ()

<img src="graph5.svg" width=500 />